Test of the new data preperation function

## Get data

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor

import math
import numpy as np

from tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt
import random

In [2]:
# Check if GPU is available, set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

In [3]:
sos_token = np.full((1, 24), 1)
pad_token = np.full((1, 24), 2)
pad_token = torch.tensor(pad_token, device=device)

dataset_dir = "/home/falaxdb/Repos/minus1/datasets/maestro_v3_split/hands_split_into_seperate_midis"
snapshot_intervall = 0.05

# Define other parameters
batch_size = 64
seq_length = 512
stride = 256

test_size=0.3

In [4]:
from transformer_decoder_training.dataprep_transformer.prepare_dataloader_complete import prepare_dataset_as_dataloaders

# Load Data

train_loader, val_loader, test_loader = prepare_dataset_as_dataloaders(dataset_dir, snapshot_intervall, batch_size, seq_length, stride, test_size, sos_token)

Processed dataset (1038/1038): 100%|██████████| 1038/1038 [00:14<00:00, 72.90it/s]


Processed 1038 of 1038 files


# initialize model

In [5]:
# set parameters
# Learning rate for the optimizer
learning_rate = 1e-3
# Number of epochs for training
nepochs = 20
# Embedding Size
hidden_size = 256
# Number of transformer blocks
num_layers = 8
# MultiheadAttention Heads
num_heads = 8

In [6]:
from transformer_decoder_training.models.transformer_decoder_2 import Transformer

model = Transformer(num_emb=24, num_layers=num_layers, hidden_size=hidden_size, num_heads=num_heads).to(device)

# Initialize the optimizer with above parameters
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Define the loss function
# loss function should be one that can handle multi one hot encoded vectors
# Klammern nicht vergessen
# Chat gpt says BCEWithLogitsLoss is more stable
loss_fn = nn.BCEWithLogitsLoss()

In [7]:
from timeit import default_timer as timer
from transformer_decoder_training.training import training_1

NUM_EPOCHS = 21

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = training_1.train_loop(model, optimizer, loss_fn, train_loader, pad_token, device)
    end_time = timer()
    val_loss = training_1.validation_loop(model, loss_fn, val_loader, pad_token, device)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))

Epoch: 1, Train loss: 0.191, Val loss: 0.129, Epoch time = 44.787s
Epoch: 2, Train loss: 0.127, Val loss: 0.125, Epoch time = 44.757s
Epoch: 3, Train loss: 0.123, Val loss: 0.122, Epoch time = 44.990s
Epoch: 4, Train loss: 0.120, Val loss: 0.120, Epoch time = 44.925s
Epoch: 5, Train loss: 0.118, Val loss: 0.118, Epoch time = 45.111s
Epoch: 6, Train loss: 0.117, Val loss: 0.117, Epoch time = 44.873s
Epoch: 7, Train loss: 0.115, Val loss: 0.117, Epoch time = 44.724s
Epoch: 8, Train loss: 0.114, Val loss: 0.114, Epoch time = 44.711s
Epoch: 9, Train loss: 0.112, Val loss: 0.112, Epoch time = 44.708s
Epoch: 10, Train loss: 0.111, Val loss: 0.110, Epoch time = 44.755s
Epoch: 11, Train loss: 0.109, Val loss: 0.109, Epoch time = 44.716s
Epoch: 12, Train loss: 0.108, Val loss: 0.108, Epoch time = 44.706s
Epoch: 13, Train loss: 0.107, Val loss: 0.107, Epoch time = 44.719s
Epoch: 14, Train loss: 0.105, Val loss: 0.106, Epoch time = 44.717s
Epoch: 15, Train loss: 0.104, Val loss: 0.105, Epoch time

In [8]:
torch.save(model.state_dict(), "/home/falaxdb/Repos/minus1/transformer_decoder_training/saved_files/saved_models/model_1_notebook_v6.1.pth")